In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import numpy as np
import pandas as pd
import pingouin as pg
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

import ensembling
import metrics
import parsing
import plotting_utils
import postprocessing

In [3]:
MAIN_PATH = '../data/outputs'
AVG_FUN = np.nanmedian  # Reduce function to compute averages across targets. Using nanmedian because there is some issue with RF/NN-score.
PROTEIN_LIGAND_PAIR_REDUCE = 'max'  # Reduce function for protein-ligand pairs multiple entries (e.g. Vina, docking, RF/NN-score etc.)
PAIR_ID_COLS = ['target_id', 'ligand_id']  # Specifies a unique protein-ligand pair

PATH_RESULTS = {
    "DENVIS-G": {
        "atom": os.path.join(MAIN_PATH, 'denvis_outputs', 'dude_main_general_atom.parquet'),
        "surface": os.path.join(MAIN_PATH, 'denvis_outputs', 'dude_main_general_surface.parquet'),},
    "DENVIS-R": {
        "atom": os.path.join(MAIN_PATH, 'newdefault_CNNscore-max.summary', 'dude_main_refined_atom.parquet'),
        "surface": os.path.join(MAIN_PATH, 'denvis_outputs', 'dude_main_refined_surface.parquet')},
    "GNINA": os.path.join(MAIN_PATH, 'newdefault_CNNscore-max.summary'),
    "Vina": MAIN_PATH,
    "RF-score": os.path.join(MAIN_PATH, 'vina_outputs', 'dude.csv'),
    "NN-score": MAIN_PATH,
    "Deep DTA": MAIN_PATH,
    "Ligand bsl.": MAIN_PATH}

# Network output combination kwargs for refined/general models
OUTPUT_COMBINATION_KWS = {
    'refined': {'y_aff_weight': 0., 'y_kd_weight': 0.5, 'y_ki_weight': 0.5, 'y_ic50_weight': 0.0, 'use_clf': False, 'clf_strategy': None},
    'general': {'y_aff_weight': 0., 'y_kd_weight': 0.5, 'y_ki_weight': 0.5, 'y_ic50_weight': 0.5, 'use_clf': False, 'clf_strategy': None},
}

In [5]:
p_ = os.path.join(MAIN_PATH, 'denvis_outputs', 'dude_main_general_atom.parquet')
parsing.parse_results_denvis(p_)

(        target_id      ligand_id  version  ckpt  y_true  y_score_Kd  \
 0            prgr  CHEMBL1164248        0     0    True    1.025230   
 1            prgr  CHEMBL1164338        0     0    True    1.281144   
 2            prgr   CHEMBL398801        0     0    True    2.119537   
 3            prgr   CHEMBL102179        0     0    True    0.984597   
 4            prgr   CHEMBL280805        0     0    True    1.856881   
 ...           ...            ...      ...   ...     ...         ...   
 7079330      ptn1   ZINC59327091        4     0   False   -0.194650   
 7079331      ptn1   ZINC62695184        4     0   False   -1.239572   
 7079332      ptn1   ZINC64502652        4     0   False    0.970383   
 7079333      ptn1   ZINC64502653        4     0   False    0.970383   
 7079334      ptn1   ZINC64819979        4     0   False    0.039873   
 
          y_score_Ki  y_score_IC50  
 0          1.043376      2.447320  
 1          0.824085      3.717463  
 2          1.101511   

In [ ]:
print(f"Parsing atom-level results (refined)...")
results_tmp, _ = parsing.parse_results_denvis(PATH_RESULTS_REFINED_ATOM, dataset=DATASET, fmt='parquet')
results_refined_atom = ensembling.compute_ensemble_scores(results_tmp, ckpt=True, version=True, pair_id_cols=PAIR_ID_COLS)

print(f"Parsing surface-level results (refined)...")
results_tmp, _ = parsing.parse_results_denvis(PATH_RESULTS_REFINED_SURFACE, dataset=DATASET, fmt='parquet')
results_refined_surface = ensembling.compute_ensemble_scores(results_tmp, ckpt=True, version=True, pair_id_cols=PAIR_ID_COLS)

print(f"Parsing ligand baseline results (refined)...")
results_tmp, _ = parsing.parse_results_denvis(PATH_RESULTS_REFINED_LIGAND, dataset=DATASET, fmt='parquet')  
results_refined_ligand = ensembling.compute_ensemble_scores(results_tmp, ckpt=True, version=True, pair_id_cols=PAIR_ID_COLS)

print(f"Parsing atom-level results (general)...")
results_tmp, _ = parsing.parse_results_denvis(PATH_RESULTS_GENERAL_ATOM, dataset=DATASET, fmt='parquet')
results_general_atom = ensembling.compute_ensemble_scores(results_tmp, ckpt=True, version=True, pair_id_cols=PAIR_ID_COLS)

print(f"Parsing surface-level results (general)...")
results_tmp, _ = parsing.parse_results_denvis(PATH_RESULTS_GENERAL_SURFACE, dataset=DATASET, fmt='parquet')
results_general_surface = ensembling.compute_ensemble_scores(results_tmp, ckpt=True, version=True, pair_id_cols=PAIR_ID_COLS)

print(f"Parsing ligand baseline results (general)...")
results_tmp, _ = parsing.parse_results_denvis(PATH_RESULTS_GENERAL_LIGAND, dataset=DATASET, fmt='parquet')  
results_general_ligand = ensembling.compute_ensemble_scores(results_tmp, ckpt=True, version=True, pair_id_cols=PAIR_ID_COLS)

print(f"Parsing DeepDTA results...")
results_deep_dta = parsing.parse_results_deeppurpose(PATH_RESULTS_DEEP_DTA, dataset=DATASET)

print(f"Parsing Vina results...")
results_vina = parsing.parse_vina(PATH_RESULTS_VINA, dataset=DATASET, reduce=PROTEIN_LIGAND_PAIR_REDUCE)    

print(f"Parsing GNINA results...")
results_gnina = parsing.parse_results_gnina(PATH_RESULTS_GNINA, dataset=DATASET)

print(f"Parsing NN-score results...")
results_nn_score = parsing.parse_results_nn_score(PATH_RESULTS_RF_NN_SCORE, dataset=DATASET, reduce=PROTEIN_LIGAND_PAIR_REDUCE, prog_bar=True)
print(f"Parsing RF-score results...")
results_rf_score = parsing.parse_results_rf_score(PATH_RESULTS_RF_NN_SCORE, dataset=DATASET, reduce=PROTEIN_LIGAND_PAIR_REDUCE, prog_bar=True)

print(f"Parsing Glide results...")
results_glide = parsing.parse_results_docking(PATH_RESULTS_DOCKING, dataset=DATASET, method='Glide', reduce=PROTEIN_LIGAND_PAIR_REDUCE)
print(f"Parsing Surflex results...")
results_surflex = parsing.parse_results_docking(PATH_RESULTS_DOCKING, dataset=DATASET, method='Surflex', reduce=PROTEIN_LIGAND_PAIR_REDUCE)
print(f"Parsing Flexx results...")
results_flexx = parsing.parse_results_docking(PATH_RESULTS_DOCKING, dataset=DATASET, method='Flexx', reduce=PROTEIN_LIGAND_PAIR_REDUCE)
print(f"Parsing Gold results...")
results_gold = parsing.parse_results_docking(PATH_RESULTS_DOCKING, dataset=DATASET, method='Gold', reduce=PROTEIN_LIGAND_PAIR_REDUCE)

print('Done.')